In [1]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 11.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.5.1
    Uninstalling fsspec-2025.5.1:
      Successfully uninstalled fsspec-2025.5.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvid

In [2]:
# =========================
# 0) Imports & basic setup
# =========================
import os, math
import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    get_linear_schedule_with_warmup
)

import evaluate
from tqdm.auto import tqdm
import wandb
# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# Reproducibility (optional)
SEED = 42
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)


wandb.login(key="017a8a1cf1968e847ba05f92a8935af78befe33f")

BASE_MODEL_OR_DIR = "digitalepidemiologylab/covid-twitter-bert"

2025-08-21 09:14:43.357304: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755767683.591491      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755767683.654665      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Device: cuda


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: rabeaotman (rabeaotman-tel-aviv-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [3]:
OLD_CKPT_DIR = "/kaggle/input/covidbertttttttttt/pytorch/default/1"   
NUM_LABELS   = 5  
IGNORE_HEAD_MISMATCH = False

tokenizer = AutoTokenizer.from_pretrained(OLD_CKPT_DIR, use_fast=True)

model = AutoModelForSequenceClassification.from_pretrained(
    OLD_CKPT_DIR,
    num_labels=NUM_LABELS,
    ignore_mismatched_sizes=IGNORE_HEAD_MISMATCH
).to(device)

In [4]:
import pandas as pd
import torch
import random, numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder   # <-- this fixes your error
from transformers import set_seed
DATA_PATH = "/kaggle/input/edaaaaa-tergol5/my_eda.csv"
TEXT_COL   = "Tweet"        # Text column after your EDA
TARGET_COL = "Sentiment"    # String labels
SEED = 42
set_seed(SEED)
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)


# --------- Load & encode labels ---------
df = pd.read_csv(DATA_PATH, encoding="latin1")
df = df.rename(columns={'Tweet': 'Original'})
df = df.rename(columns={'normalized_tweet': 'Tweet'})  # this becomes the text we train on
assert TEXT_COL in df.columns and TARGET_COL in df.columns, f"Missing {TEXT_COL}/{TARGET_COL} in CSV."


le = LabelEncoder()
df["label"] = le.fit_transform(df[TARGET_COL])
num_labels = df["label"].nunique()
print("Labels:", dict(enumerate(le.classes_)))
print("num_labels =", num_labels)

# Stratified split
train_df, eval_df = train_test_split(
    df, test_size=0.2, random_state=SEED, stratify=df["label"]
)
len(train_df), len(eval_df)

Labels: {0: 'Extremely Negative', 1: 'Extremely Positive', 2: 'Negative', 3: 'Neutral', 4: 'Positive'}
num_labels = 5


(32873, 8219)

In [5]:
######bessssssttttttt paraaaaaaaaaams

best_params = {'learning_rate': 1.5252567328044727e-05,
'weight_decay': 5.256377166554879e-06,
'warmup_ratio': 0.13130278878798132,
'max_length': 96,
'train_bs': 16,
'grad_acc': 1,
'label_smoothing': 0.02939616441539268,
'gradient_checkpointing': True}


In [6]:
from datasets import Dataset, DatasetDict

# =========================
# 2) Build HF DatasetDict
# =========================
# Keep only the columns we need for training: text + numeric label
cols = [TEXT_COL, "label"]
train_ds = Dataset.from_pandas(train_df[cols].rename(columns={"label": "labels"}), preserve_index=False)
eval_ds  = Dataset.from_pandas(eval_df[cols].rename(columns={"label": "labels"}),  preserve_index=False)

# Use keys that match your earlier code (train/test)
raw_datasets = DatasetDict({"train": train_ds, "test": eval_ds})
raw_datasets

# =========================
# 3) Tokenizer & tokenization
# =========================
tokenizer = AutoTokenizer.from_pretrained(OLD_CKPT_DIR, use_fast=True)

def tokenize_fn(batch):
    return tokenizer(
        batch[TEXT_COL],
        truncation=True,
        max_length=best_params['max_length'],   # or remove to use dynamic truncation
    )

tokenized_datasets = raw_datasets.map(
    tokenize_fn,
    batched=True,
    remove_columns=[TEXT_COL],   # drop raw text; keep only tokenized fields + "labels"
)

# Make sure the column is named exactly "labels" (HF models expect this)
assert "labels" in tokenized_datasets["train"].column_names

# Set torch format for PyTorch DataLoader downstream
tokenized_datasets.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "labels"],
)
tokenized_datasets


Map:   0%|          | 0/32873 [00:00<?, ? examples/s]

Map:   0%|          | 0/8219 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 32873
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 8219
    })
})

In [7]:
from transformers import set_seed, AutoTokenizer, DataCollatorWithPadding
# =========================
# 4) Dataloaders (same pattern you use)
# =========================
from torch.utils.data import DataLoader

# Optional: pad to multiples of 8 for better Tensor Core utilization on GPUs
collator = DataCollatorWithPadding(
    tokenizer=tokenizer,
    pad_to_multiple_of=8 if torch.cuda.is_available() else None
)

# Small subsets exactly like your snippet
train_dataset = tokenized_datasets["train"].shuffle(seed=SEED)


eval_dataset  = tokenized_datasets["test"].shuffle(seed=SEED)

train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=best_params['train_bs'], collate_fn=collator)
eval_dataloader  = DataLoader(eval_dataset, batch_size=best_params['train_bs'], collate_fn=collator)




In [8]:
# ============================================
# 4) Optimizer & Scheduler (same pattern)
# ============================================
learning_rate = best_params['learning_rate']
num_epochs = 3

optimizer = AdamW(model.parameters(), lr=learning_rate, weight_decay = best_params["weight_decay"])

num_training_steps = num_epochs * len(train_dataloader)

# You can add warmup steps if you like (e.g., 10% of total steps)
warmup_steps = int(best_params["warmup_ratio"] * num_training_steps)

lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=warmup_steps,
    num_training_steps=num_training_steps
)


In [9]:
# =========================
# Train → Eval → Save + W&B
# =========================
import os, time
from tqdm.auto import tqdm
import evaluate
import torch


# ----- W&B setup -----
WANDB_PROJECT = "finetune-hf-ct-bert"   # <-- change if you want
RUN_NAME      = "last take"

# If your API key is in the env (recommended on Kaggle), this will pick it up automatically.
wandb.login()
wandb.init(project=WANDB_PROJECT, name=RUN_NAME, config={
    "num_epochs": num_epochs,
    "train_batch_size": train_dataloader.batch_size,
    "eval_batch_size": eval_dataloader.batch_size,
    "max_steps": num_training_steps,
    "model_name_or_path": (MODEL_DIR if 'MODEL_DIR' in globals() else "unknown"),
})

# Optional: track gradients/weights; set log_freq to every N steps
wandb.watch(model, log="gradients", log_freq=50)

scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available())
progress_bar = tqdm(range(num_training_steps))

global_step = 0
model.train()
for epoch in range(num_epochs):
    epoch_loss = 0.0
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}

        optimizer.zero_grad(set_to_none=True)

        with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):
            outputs = model(**batch)
            loss = outputs.loss

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        lr_scheduler.step()

        # --- Logging per step ---
        lr_now = lr_scheduler.get_last_lr()[0] if hasattr(lr_scheduler, "get_last_lr") else optimizer.param_groups[0]["lr"]
        wandb.log({
            "train/step_loss": float(loss.item()),
            "train/lr": float(lr_now),
            "train/epoch": epoch,
            "train/global_step": global_step,
        }, step=global_step)

        epoch_loss += loss.item()
        progress_bar.update(1)
        global_step += 1

    avg = epoch_loss / max(1, len(train_dataloader))
    print(f"Epoch {epoch+1}/{num_epochs} - train_loss: {avg:.4f}")
    # --- Logging per epoch ---
    wandb.log({"train/epoch_loss": float(avg), "train/epoch_end": epoch}, step=global_step)

# =========================
# Evaluation (accuracy)
# =========================
metric = evaluate.load("accuracy")
model.eval()
eval_loss = 0.0

with torch.no_grad():
    for batch in eval_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)

        # get predictions
        preds = torch.argmax(outputs.logits, dim=-1)
        metric.add_batch(predictions=preds, references=batch["labels"])

        # accumulate loss
        eval_loss += outputs.loss.item()

# average loss
avg_eval_loss = eval_loss / max(1, len(eval_dataloader))

# accuracy
acc = metric.compute()  # {'accuracy': ...}

print(f"Final Eval Accuracy: {acc['accuracy']:.4f}")
print(f"Final Eval Loss: {avg_eval_loss:.4f}")

wandb.log({
    "eval/accuracy": float(acc["accuracy"]),
    "eval/loss": float(avg_eval_loss)
}, step=global_step)


# =========================
# Save checkpoint (local)
# =========================
SAVE_DIR = "/kaggle/working/finetuned_from_old"
os.makedirs(SAVE_DIR, exist_ok=True)
model.save_pretrained(SAVE_DIR, safe_serialization=True)
tokenizer.save_pretrained(SAVE_DIR)
print("Model & tokenizer saved to:", SAVE_DIR)

# =========================
# Log model as a W&B artifact
# =========================
artifact = wandb.Artifact(
    name="finetuned-from-old",
    type="model",
    metadata={"accuracy": float(acc["accuracy"]), "epochs": num_epochs}
)
artifact.add_dir(SAVE_DIR)
wandb.log_artifact(artifact)

wandb.finish()


/tmp/ipykernel_36/2473381169.py:27: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available())


  0%|          | 0/6165 [00:00<?, ?it/s]

/tmp/ipykernel_36/2473381169.py:39: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):


Epoch 1/3 - train_loss: 0.2405
Epoch 2/3 - train_loss: 0.1651
Epoch 3/3 - train_loss: 0.0766


Final Eval Accuracy: 0.8884
Final Eval Loss: 0.4125


wandb: Adding directory to artifact (/kaggle/working/finetuned_from_old)... 

Model & tokenizer saved to: /kaggle/working/finetuned_from_old


Done. 4.6s


eval/accuracy,▁
eval/loss,▁
train/epoch,▁▁▁▁▁▁▁▁▁▁▁▅▅▅▅▅▅▅▅▅▅▅▅▅████████████████
train/epoch_end,▁▅█
train/epoch_loss,█▅▁
train/global_step,▁▁▁▂▂▂▂▂▂▂▂▂▃▃▃▃▃▄▄▄▅▅▅▅▅▅▅▅▅▆▆▆▇▇▇▇▇███
train/lr,▂▂▄▄▅██▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▅▅▅▄▄▄▃▃▃▃▃▂▂▁▁▁▁
train/step_loss,▄▅▂▂▂▂▂▆▂▆█▅▃▇▃▃▁▂▃▂▂▅▁▂▃▅▁▂▂▁▁▃▂▂▁▁▁▁▁▁
eval/accuracy,0.88843
eval/loss,0.41252
train/epoch,2


In [10]:
# Go to the parent directory
!cd /kaggle/working && zip -r finetuned_from_old.zip finetuned_from_old

# This creates /kaggle/working/finetuned_from_old.zip


  adding: finetuned_from_old/ (stored 0%)
  adding: finetuned_from_old/config.json (deflated 54%)
  adding: finetuned_from_old/model.safetensors

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


 (deflated 7%)
  adding: finetuned_from_old/vocab.txt (deflated 53%)
  adding: finetuned_from_old/special_tokens_map.json (deflated 80%)
  adding: finetuned_from_old/tokenizer_config.json (deflated 74%)
  adding: finetuned_from_old/tokenizer.json (deflated 71%)
